# Writing High-Frequency Trajectories When an Event is Detected

In this tutorial, we will showcase how Dupin's online capabilities can be used to
trigger HOOMD's burst writer to dump high-resolution trajectory data only during
specific events detected by Dupin. This approach has significant implications for
performance and storage. The main advantage is that it allows us to write to disk only
the segments of the trajectory that are of interest, at an extremely high frequency,
without dumping the entire trajectory.

To achieve this, we create a temporary buffer that stores trajectory frames in memory.
When Dupin triggers, this buffer is dumped onto the disk. Online detection requires some
CPU cycles to run periodically, which can impact performance of the MD program depending
on the order parameters used for detection. In practice, it is often sufficient to track
very simple properties that are computed by the MD driver (in this case, HOOMD) anyway,
such as pressure, total system energy, or volume (depending on whether we are running
NVT/NVE or NPT ensembles).

Here, we provide an example:

In [ ]:
import hoomd
import numpy as np
import ruptures as rpt

import dupin as du

cpu = hoomd.device.CPU()
simulation = hoomd.Simulation(cpu, seed=1)

# Create a simple cubic configuration of particles
N = 10  # particles per box direction
box_l = 13  # box dimension

snap = hoomd.Snapshot(cpu.communicator)
snap.configuration.box = [box_l] * 3 + [0, 0, 0]
snap.particles.N = N**3
x, y, z = np.meshgrid(
    *(np.linspace(-box_l / 2, box_l / 2, N, endpoint=False),) * 3
)
positions = np.array((x.ravel(), y.ravel(), z.ravel())).T
snap.particles.position[:] = positions
snap.particles.types = ["A"]
snap.particles.typeid[:] = 0

simulation.create_state_from_snapshot(snap)

burst_filename = "test.gsd"

burst = hoomd.write.Burst(
    trigger=hoomd.trigger.Periodic(100),
    filename=burst_filename,
    max_burst_size=10000,
    write_at_start=True,
)
simulation.operations.writers.append(burst)

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell(buffer=0.4)
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[("A", "A")] = dict(epsilon=1, sigma=1)
lj.r_cut[("A", "A")] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.ConstantVolume(
    filter=hoomd.filter.All(),
    thermostat=hoomd.md.methods.thermostats.Bussi(kT=1.5),
)
integrator.methods.append(nvt)

thermo = hoomd.md.compute.ThermodynamicQuantities(hoomd.filter.All())
simulation.operations += thermo


def custom_dupin_generator_fn(data_du):
    return {"energy": data_du}


inital_frames = 15


class OnlineWriter(hoomd.custom.Action):
    def __init__(self, burst_writer, dupin_generator, thermo):
        self.burst_writer = burst_writer
        self.custom_generator = dupin_generator
        self.create_aggregator()
        self.thermo = thermo
        lin_regress_cost = du.detect.CostLinearFit()
        dynp = rpt.Dynp(custom_cost=lin_regress_cost)
        self.sweep_detector = du.detect.SweepDetector(dynp, max_change_points=6)

    def act(self):
        data = self.thermo["potential_energy"]
        self.signal_aggregator.accumulate(data)
        #### Also should skip detection depending on how many data points signal has, if it has 1 or two points it wont make any sense.
        # 15 is a random number here, use something better?
        # not sure if len is the correct way to figure out how many data points are there in the signal
        if len(self.signal_aggregator.signals) > inital_frames:
            if (
                len(
                    self.sweep_detector.fit(
                        self.signal_aggregator.to_dataframe()
                    )
                )
                > 0
            ):
                self.burst_writer.dump()
                self.create_aggregator()

    def create_aggregator(self, dupin_generator):
        custom_generator = du.data.base.CustomGenerator(self.custom_generator)
        self.signal_aggregator = du.data.aggregate.SignalAggregator(
            custom_generator
        )


dupin_online_writer = hoomd.write.CustomWriter(
    action=OnlineWriter(burst, custom_dupin_generator_fn, thermo), trigger=1000
)
simulation.operations += dupin_online_writer

simulation.run(10000)